In [1]:
import re
import os
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import time

try:
    import FinanceDataReader as fdr
    from pykrx import stock
except:
    os.system('pip install finance-datareader')
    os.system('pip install pykrx')
    import FinanceDataReader as fdr
    from pykrx import stock

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
except:
    os.system('pip install plotly')
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    
# pandas copy error 끄기
pd.set_option('mode.chained_assignment',  None)
pd.options.display.float_format = '{:.2f}'.format

# 파일 경로 설정
path = os.getcwd()
clas_path = path +'\\Classification\\'
img_path = path +'\\images\\'

In [2]:
# 날짜 설정
print("입력예시: 2020-01-01")
previous = str(input("이전 거래일을 입력하세요: ").replace('-', ''))
today = str(input("오늘 날짜를 입력하세요: ").replace('-', ''))

# 표준산업분류 불러오기
classification = pd.read_csv(clas_path +'\classification.csv', usecols=['L1','L2','L3'])

# 코스피 종목 정보 불러오기
kospi_list = fdr.StockListing('KOSPI')
    # 우선주, 투자신탁 제거
kospi_list = kospi_list.dropna(axis=0).reset_index(drop=True)
    # 코스피 종목 정보 추리기
kospi_info= pd.DataFrame(kospi_list, columns = ['Symbol', 'Name', 'Sector'])


# 상위 산업 추가하기

# 띄워쓰기가 다른 경우가 있음 -> 띄워쓰기 전부 제거
# 특수 문자 모두 제거

for idx, row in classification.iterrows():
    no_space = row['L3'].replace(' ', '')
    row['L3'] = no_space

    no_specials = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', row['L3'])
    row['L3'] = no_specials

# 코스피 상위 산업 추가하기

kospi_info['L2'] = None
kospi_info['L1'] = None

for idx, row in kospi_info.iterrows():
    
    sector = row['Sector'].replace(' ', '') # 코스피의 섹터도 동일하게 띄워쓰기 제거
    sector = re.sub('[-=+,#/·\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sector) # 모든 특수문자 제거

    L1 = list(classification[classification['L3'] == sector]['L1'])
    L2 = list(classification[classification['L3']== sector]['L2'])
    
    if len(L1) > 0:
        kospi_info['L1'][idx] = L1[0]
        kospi_info['L2'][idx] = L2[0]
    else:
        print(sector)

입력예시: 2020-01-01


In [37]:
# 컬럼명 바꾸기
kospi_info = kospi_info.rename(columns={'Symbol':'Ticker', 'Sector': 'L3'})

kospi_ohlcv_pre = stock.get_market_ohlcv_by_ticker(previous, "KOSPI").reset_index(drop=False)
kospi_ohlcv_pre = kospi_ohlcv_pre.rename(
    columns = {
        '티커':'Ticker',
        '종목명':'Name',
        '시가': 'Open',
        '고가': 'High',
        '저가': 'Low',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume'
         }
    )

kospi_ohlcv_today = stock.get_market_ohlcv_by_ticker(today, "KOSPI").reset_index(drop=False)
kospi_ohlcv_today = kospi_ohlcv_today.rename(
    columns = {
        '티커':'Ticker',
        '종목명':'Name',
        '시가': 'Open',
        '고가': 'High',
        '저가': 'Low',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume'
         }
    )


market_cap_pre = stock.get_market_cap_by_ticker(previous, "KOSPI").reset_index(drop=False)

market_cap_pre = market_cap_pre.rename(
    columns = {
        '티커':'Ticker',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume',
        '시가총액': 'Market Cap',
        '상장주식수': 'Share Outstanding'
         }
    )


market_cap_today = stock.get_market_cap_by_ticker(today, "KOSPI").reset_index(drop=False)

market_cap_today = market_cap_today.rename(
    columns = {
        '티커':'Ticker',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume',
        '시가총액': 'Market Cap',
        '상장주식수': 'Share Outstanding'
         }
    )



# 코스피 데이터 병합
kospi_info['Open'] = None
kospi_info['Close'] = None
kospi_info['Pr_Change'] = None
kospi_info['Change'] = None
kospi_info['MarCap_pre'] = None
kospi_info['MarCap_today'] = None
kospi_info['sqrtMarCap'] = None
kospi_info['Status'] = None

for idx, row in kospi_info.iterrows():
    ticker = row['Ticker']
    
    stock_ohlcv_today = kospi_ohlcv_today[kospi_ohlcv_today['Ticker'] == ticker]

    mcap_td = market_cap_today[market_cap_today["Ticker"] == ticker]['Market Cap'].iloc[0]  
    row['MarCap_today'] = int(mcap_td)

    mcap_pr = market_cap_pre[market_cap_pre["Ticker"] == ticker]['Market Cap'].iloc[0] 
    row['MarCap_pre'] = int(mcap_pr)
       
    if stock_ohlcv_today['Open'].iloc[0] != 0: # 거래정지가 아닌 경우
        row['Open'] = stock_ohlcv_today['Open'].iloc[0]
        row['Close'] = stock_ohlcv_today['Close'].iloc[0]
        
        # 전일 대비 가격 변동
        pre_close = kospi_ohlcv_pre[kospi_ohlcv_pre['Ticker'] == ticker]['Close'].iloc[0]
        ch = row['Close'] - pre_close
        row['Pr_Change'] = ch
        pch = round((row['Close'] - pre_close) / pre_close * 100, 2)
        row['Change'] = pch
        
        row['sqrtMarCap'] = np.sqrt(int(mcap_td))
        row['Status'] = 'Active'

    else: # 거래정지
        row['Open'] = stock_ohlcv_today['Close'].iloc[0]
        row['Close'] = stock_ohlcv_today['Close'].iloc[0]
        row['Change'] = 0
        
        row['sqrtMarCap'] = np.sqrt(int(mcap_td))
        row['Status'] = 'Suspend'

kospi_info['Market'] = 'KOSPI'
kospi_info['Open'] = kospi_info['Open'].astype(float)
kospi_info['Close'] = kospi_info['Close'].astype(float)
kospi_info['Change'] = kospi_info['Change'].astype(float)
kospi_info['MarCap_pre'] = kospi_info['MarCap_pre'].astype(float)
kospi_info['MarCap_today'] = kospi_info['MarCap_today'].astype(float)
kospi_info['sqrtMarCap'] = kospi_info['sqrtMarCap'].astype(float)

In [38]:
kospi_info

,Ticker,Name,L3,L2,L1,Open,Close,Pr_Change,Change,MarCap_pre,MarCap_today,sqrtMarCap,Status,parent,Market
0,095570,AJ네트웍스,산업용 기계 및 장비 임대업,임대업,"사업시설 관리, 사업 지원 및 임대 서비스업",4190.00,4210.00,35,0.84,195483081625.00,197121861950.00,443984.08,Active,산업용 기계 및 장비 임대업,KOSPI
1,006840,AK홀딩스,기타 금융업,금융업,금융 및 보험업,25750.00,29100.00,3250,12.57,342449451850.00,385504025100.00,620889.70,Active,기타 금융업,KOSPI
2,027410,BGF,기타 금융업,금융업,금융 및 보험업,5020.00,5220.00,210,4.19,479541122910.00,499641649020.00,706853.34,Active,기타 금융업,KOSPI
3,282330,BGF리테일,종합 소매업,소매업,도매 및 소매업,156500.00,152000.00,-4000,-2.56,2696289336000.00,2627153712000.00,1620849.69,Active,종합 소매업,KOSPI
4,138930,BNK금융지주,기타 금융업,금융업,금융 및 보험업,5720.00,5610.00,-120,-2.09,1867608959580.00,1828496730060.00,1352219.19,Active,기타 금융업,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,079980,휴비스,화학섬유 제조업,화학 물질 및 화학제품 제조업,제조업,8080.00,8060.00,10,0.12,277725000000.00,278070000000.00,527323.43,Active,화학섬유 제조업,KOSPI
760,005010,휴스틸,1차 철강 제조업,1차 금속 제조업,제조업,7990.00,8010.00,100,1.26,60287393880.00,61049560680.00,247082.09,Active,1차 철강 제조업,KOSPI
761,069260,휴켐스,기타 화학제품 제조업,화학 물질 및 화학제품 제조업,제조업,23950.00,23650.00,-50,-0.21,968822535600.00,966778606200.00,983249.01,Active,기타 화학제품 제조업,KOSPI
762,000540,흥국화재,보험업,보험 및 연금업,금융 및 보험업,3390.00,3310.00,-105,-3.07,219388632675.00,212643154950.00,461132.47,Active,보험업,KOSPI


In [57]:

fig = px.treemap(
    kospi_info,
    path = ['Market','L1', 'L2', 'Name'],
    values = 'sqrtMarCap',
    color = 'Change',
    color_continuous_scale= [[0, '#14029e'], [0.5, '#424242'], [1, '#8c0618']],
    color_continuous_midpoint = 0,
    range_color = [-3,3],
    branchvalues = 'total',
    custom_data = ['Change'],
    maxdepth=5
)

fig.update_traces(
    textposition = 'middle center',
    marker_line_width= 0.2,
    hovertemplate = '<b>%{label}</b>',
    texttemplate = '%{label}<br><br>%{customdata[0]:.2f}%'
    )

fig.update_layout(
    autosize = False,
    width = 1080,
    height = 640,
    margin = dict(l=0, r=0, t=0, b=0),
    coloraxis_showscale = False
)


In [6]:
# 코스피 종목 정보 불러오기
kosdaq_list = fdr.StockListing('KOSDAQ')
    # 우선주, 투자신탁 제거
kosdaq_list = kosdaq_list.dropna(axis=0).reset_index(drop=True)
    # 코스피 종목 정보 추리기
kosdaq_info= pd.DataFrame(kosdaq_list, columns = ['Symbol', 'Name', 'Sector'])


# 상위 산업 추가하기

# 띄워쓰기가 다른 경우가 있음 -> 띄워쓰기 전부 제거
# 특수 문자 모두 제거

for idx, row in classification.iterrows():
    no_space = row['L3'].replace(' ', '')
    row['L3'] = no_space

    no_specials = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', row['L3'])
    row['L3'] = no_specials

# 코스피 상위 산업 추가하기

kosdaq_info['L2'] = None
kosdaq_info['L1'] = None

for idx, row in kosdaq_info.iterrows():
    
    sector = row['Sector'].replace(' ', '') # 코스피의 섹터도 동일하게 띄워쓰기 제거
    sector = re.sub('[-=+,#/·\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sector) # 모든 특수문자 제거

    L1 = list(classification[classification['L3'] == sector]['L1'])
    L2 = list(classification[classification['L3']== sector]['L2'])
    
    if len(L1) > 0:
        kosdaq_info['L1'][idx] = L1[0]
        kosdaq_info['L2'][idx] = L2[0]
    else:
        print(sector)

# 컬럼명 바꾸기
kosdaq_info = kosdaq_info.rename(columns={'Symbol':'Ticker', 'Sector': 'L3'})

kosdaq_ohlcv_pre = stock.get_market_ohlcv_by_ticker(previous, "KOSDAQ").reset_index(drop=False)
kosdaq_ohlcv_pre = kosdaq_ohlcv_pre.rename(
    columns = {
        '티커':'Ticker',
        '종목명':'Name',
        '시가': 'Open',
        '고가': 'High',
        '저가': 'Low',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume'
         }
    )

kosdaq_ohlcv_today = stock.get_market_ohlcv_by_ticker(today, "KOSDAQ").reset_index(drop=False)
kosdaq_ohlcv_today = kosdaq_ohlcv_today.rename(
    columns = {
        '티커':'Ticker',
        '종목명':'Name',
        '시가': 'Open',
        '고가': 'High',
        '저가': 'Low',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume'
         }
    )


market_cap_pre = stock.get_market_cap_by_ticker(previous, "KOSDAQ").reset_index(drop=False)

market_cap_pre = market_cap_pre.rename(
    columns = {
        '티커':'Ticker',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume',
        '시가총액': 'Market Cap',
        '상장주식수': 'Share Outstanding'
         }
    )


market_cap_today = stock.get_market_cap_by_ticker(today, "KOSDAQ").reset_index(drop=False)

market_cap_today = market_cap_today.rename(
    columns = {
        '티커':'Ticker',
        '종가': 'Close',
        '거래량': 'Volume',
        '거래대금': 'Transaction Volume',
        '시가총액': 'Market Cap',
        '상장주식수': 'Share Outstanding'
         }
    )



# 코스피 데이터 병합
kosdaq_info['Open'] = None
kosdaq_info['Close'] = None
kosdaq_info['Pr_Change'] = None
kosdaq_info['Change'] = None
kosdaq_info['MarCap_pre'] = None
kosdaq_info['MarCap_today'] = None
kosdaq_info['sqrtMarCap'] = None
kosdaq_info['Status'] = None

for idx, row in kosdaq_info.iterrows():
    ticker = row['Ticker']
    
    stock_ohlcv_today = kosdaq_ohlcv_today[kosdaq_ohlcv_today['Ticker'] == ticker]

    mcap_td = market_cap_today[market_cap_today["Ticker"] == ticker]['Market Cap'].iloc[0]  
    row['MarCap_today'] = int(mcap_td)

    mcap_pr = market_cap_pre[market_cap_pre["Ticker"] == ticker]['Market Cap'].iloc[0] 
    row['MarCap_pre'] = int(mcap_pr)
       
    if stock_ohlcv_today['Open'].iloc[0] != 0: # 거래정지가 아닌 경우
        row['Open'] = stock_ohlcv_today['Open'].iloc[0]
        row['Close'] = stock_ohlcv_today['Close'].iloc[0]
        
        # 전일 대비 가격 변동
        pre_close = kosdaq_ohlcv_pre[kosdaq_ohlcv_pre['Ticker'] == ticker]['Close'].iloc[0]
        ch = row['Close'] - pre_close
        row['Pr_Change'] = ch
        pch = round((row['Close'] - pre_close) / pre_close * 100, 2)
        row['Change'] = pch
        
        row['sqrtMarCap'] = np.sqrt(int(mcap_td))
        row['Status'] = 'Active'

    else: # 거래정지
        row['Open'] = stock_ohlcv_today['Close'].iloc[0]
        row['Close'] = stock_ohlcv_today['Close'].iloc[0]
        row['Change'] = 0
        
        row['sqrtMarCap'] = np.sqrt(int(mcap_td))
        row['Status'] = 'Suspend'

kosdaq_info['Market'] = 'KOSDAQ'
kosdaq_info['Open'] = kosdaq_info['Open'].astype(float)
kosdaq_info['Close'] = kosdaq_info['Close'].astype(float)
kosdaq_info['Change'] = kosdaq_info['Change'].astype(float)
kosdaq_info['MarCap_pre'] = kosdaq_info['MarCap_pre'].astype(float)
kosdaq_info['MarCap_today'] = kosdaq_info['MarCap_today'].astype(float)
kosdaq_info['sqrtMarCap'] = kosdaq_info['sqrtMarCap'].astype(float)

fig = px.treemap(
    kosdaq_info,
    path = ['Market','L1', 'L2', 'Name'],
    values = 'MarCap_today',
    color = 'Change',
    color_continuous_scale= [[0, '#14029e'], [0.5, '#424242'], [1, '#8c0618']],
    color_continuous_midpoint = 0,
    range_color = [-3,3],
    branchvalues = 'total',
    custom_data = ['Change'],
    maxdepth=5
)

fig.update_traces(
    textposition = 'middle center',
    marker_line_width= 0.2,
    hovertemplate = '<b>%{label}</b>',
    texttemplate = '%{label}<br><br>%{customdata[0]:.2f}%'
    )

fig.update_layout(
    autosize = False,
    width = 1080,
    height = 640,
    margin = dict(l=0, r=0, t=0, b=0),
    coloraxis_showscale = False
)


